# Getting Started with LLM Lab

This notebook demonstrates the basic usage of LLM Lab for working with multiple LLM providers.

## Table of Contents
1. [Setup and Installation](#setup)
2. [Basic Provider Usage](#basic-provider)
3. [Comparing Multiple Models](#comparing-models)
4. [Cost Analysis](#cost-analysis)
5. [Next Steps](#next-steps)

## Setup and Installation <a id='setup'></a>

First, let's set up the environment and import necessary modules.

In [ ]:
# Import required libraries
import os
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import LLM Lab modules
from src.analysis.comparator import ModelComparator
from src.cost.tracker import CostTracker
from src.providers import AnthropicProvider, GoogleProvider, OpenAIProvider

print("LLM Lab modules imported successfully!")

## Basic Provider Usage <a id='basic-provider'></a>

Let's start with a simple example using the OpenAI provider.

In [ ]:
# Initialize OpenAI provider (replace with your API key)
openai_provider = OpenAIProvider(
    api_key=os.getenv("OPENAI_API_KEY", "your-api-key-here"), model="gpt-3.5-turbo", temperature=0.7
)

# Generate a simple response
prompt = "Explain machine learning in 2 sentences."
response = openai_provider.generate(prompt, max_tokens=100)

print("Model Response:")
print("-" * 50)
print(response.get("content", "No response"))
print("-" * 50)
print(f"Tokens used: {response.get('usage', {}).get('total_tokens', 'N/A')}")

## Comparing Multiple Models <a id='comparing-models'></a>

Now let's compare responses from different providers on the same prompt.

In [ ]:
# Set up multiple providers
providers = {}

# Add OpenAI if API key is available
if os.getenv("OPENAI_API_KEY"):
    providers["OpenAI (GPT-3.5)"] = OpenAIProvider(
        api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo"
    )

# Add Anthropic if API key is available
if os.getenv("ANTHROPIC_API_KEY"):
    providers["Anthropic (Claude)"] = AnthropicProvider(
        api_key=os.getenv("ANTHROPIC_API_KEY"), model="claude-3-haiku-20240307"
    )

# Add Google if API key is available
if os.getenv("GOOGLE_API_KEY"):
    providers["Google (Gemini)"] = GoogleProvider(
        api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-1.5-flash"
    )

print(f"Available providers: {list(providers.keys())}")

In [ ]:
# Compare responses on a common prompt
test_prompt = "What are the main differences between supervised and unsupervised learning?"

responses = {}
for name, provider in providers.items():
    try:
        response = provider.generate(test_prompt, max_tokens=150)
        responses[name] = {
            "content": response.get("content", "No response"),
            "tokens": response.get("usage", {}).get("total_tokens", 0),
        }
    except Exception as e:
        responses[name] = {"content": f"Error: {str(e)}", "tokens": 0}

# Display responses
for name, resp in responses.items():
    print(f"\n{'=' * 60}")
    print(f"{name}")
    print(f"{'=' * 60}")
    print(resp["content"][:300] + "..." if len(resp["content"]) > 300 else resp["content"])
    print(f"\nTokens used: {resp['tokens']}")

## Cost Analysis <a id='cost-analysis'></a>

Let's analyze the cost of our API calls.

In [ ]:
# Initialize cost tracker
cost_tracker = CostTracker()

# Simulate some API calls with cost tracking
test_prompts = ["What is Python?", "Explain neural networks.", "How does gradient descent work?"]

total_costs = {}
for name, provider in providers.items():
    provider_cost = 0.0
    for prompt in test_prompts:
        try:
            response = provider.generate(prompt, max_tokens=100)
            # Estimate cost (simplified - actual implementation would use provider-specific pricing)
            tokens = response.get("usage", {}).get("total_tokens", 0)
            cost = cost_tracker.estimate_cost(provider.provider_name, provider.model_name, tokens)
            provider_cost += cost
        except:
            pass
    total_costs[name] = provider_cost

# Display cost analysis
print("Cost Analysis Summary")
print("=" * 40)
for name, cost in total_costs.items():
    print(f"{name}: ${cost:.4f}")
print(f"\nTotal cost: ${sum(total_costs.values()):.4f}")

## Visualization of Results

Let's create a simple visualization of token usage across providers.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Prepare data for visualization
provider_names = list(responses.keys())
token_counts = [resp["tokens"] for resp in responses.values()]

# Create bar chart
if provider_names:
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.Set3(np.linspace(0, 1, len(provider_names)))
    bars = ax.bar(provider_names, token_counts, color=colors)

    ax.set_ylabel("Tokens Used")
    ax.set_title("Token Usage Comparison Across Providers")
    ax.set_xlabel("Provider")

    # Add value labels on bars
    for bar, count in zip(bars, token_counts):
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.0, height, f"{int(count)}", ha="center", va="bottom"
        )

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization. Please configure API keys.")

## Next Steps <a id='next-steps'></a>

Now that you've learned the basics, you can:

1. **Run Benchmarks**: See the `02_benchmarking.ipynb` notebook
2. **Custom Evaluations**: Check out `03_custom_evaluation.ipynb`
3. **Fine-Tuning**: Explore `04_fine_tuning_workflow.ipynb`
4. **Monitoring**: Learn about monitoring in `05_monitoring_setup.ipynb`

### Key Takeaways

- LLM Lab provides a unified interface for multiple LLM providers
- Easy comparison of responses across different models
- Built-in cost tracking and analysis
- Extensible framework for custom evaluations

## Clean Up

Don't forget to clean up resources if needed.

In [ ]:
# Clean up (if needed)
print("Notebook completed successfully!")
print(f"Total providers tested: {len(providers)}")
print(f"Total prompts processed: {len(test_prompts) * len(providers)}")